In [1]:
from sklearn.preprocessing import StandardScaler
import random as rd
from random import  uniform
from random import randint
from random import random
from operator import itemgetter
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from pandas import *
from sklearn.svm import SVR
from sklearn.svm import SVC
from sklearn import preprocessing
import numpy as np


In [2]:
#Ingreso del csv
dataset = read_csv("proyecto_pre_procesado.csv",sep=";")

#Seperacion de variables
x = dataset.iloc[:,:-1].values # DEPENDIENTES
y = dataset.iloc[:,-1:].values.flatten() #INDEPENDIENTES

#Definimos nuestras variables de entreno y prueba en 'x' y 'y'
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

#Generamos un modelo de Maquina de vectores de soporte
model = SVR(kernel='rbf', C = 500, epsilon=0.1)
model.fit(x_train, y_train)

#Verificamos el score
model.score(x_test,y_test)


-0.07796699716123578

In [15]:
#Segunda SVC
from sklearn.svm import SVC
Sclf =SVC(C=3.0, kernel='linear', degree=2, gamma='scale',max_iter=-1,probability=True)
model=Sclf.fit(x_train,y_train)
model.score(x_test,y_test)

0.26666666666666666

In [48]:
from sklearn import linear_model
from sklearn.metrics import r2_score
model=linear_model.LinearRegression()
model.fit(x_train,y_train)
model.score(x_test,y_test)

-0.8226159261375472

In [4]:

class AG:
    # poblacion,max_poblacion,min_poblacion,fitness,mutacion,p_mutacion,p_cruze
    # Minimos = [0.22,1.21,0.01,0.0027,0.1,0.01,0.01,0.01,0.0001,0.03,1,0.01,0.0012,0.01,0.01,0]
    # Maximos = [254,1.33,23,8,148,18,14,0.05,0.0005,51,57,0.11,4,0.05,15,8]
    
    def __init__(self,N=10): 
        self.tamaño_poblacion=N
        self.max_poblacion=[254,1.33,23,8,148,18,14,0.05,0.0005,51,57,0.11,4,0.05,15,8]
        self.min_poblacion=[0.22,1.21,0.01,0.0027,0.1,0.01,0.01,0.01,0.0001,0.03,1,0.01,0.0012,0.01,0.01,0]
        self.poblacion=None
        self._fitness=None
        self.mutacion=None
        self.p_mutacion=None
        self.p_cruce=None
        self.seleccion=None
        self.modelo=None
        self._padre=None
        self._madre=None
        self._hijos=None
    
    def __str__(self):
        print("Tamaño:",self.tamaño_poblacion)
        print("Max_valores:",self.max_poblacion)
        print("Min_valores:",self.min_poblacion)
        print("Poblacion:",self.poblacion)
        print("Fitness:",self.fitness)
        print("Mutacion:",self.mutacion)
        print("P_mutacion:",self.p_mutacion)
        print("P_cruze:",self.p_cruze)
        print("Modelo:",self.modelo)
        return ""

        #Poblacion= [PC;PMg;PP;PS;Psi;Pti;PNb;PCr;PBr;Pal;PV;Pcu;Pca;Pni;Pmo;Pn] [0,1] 0.2  0.03
    def _Individuo(self):
        return [rd.uniform(self.min_poblacion[i],self.max_poblacion[i]) for i in range(16)]

    def _Poblacion(self):
        self.poblacion=[self._Individuo() for i in range(self.tamaño_poblacion)]

    def _Fitness(self,individuo):
        Fitness_valor=self._Funcion_Objetivo(individuo)
        return Fitness_valor[0]

    def _Mutacion(self,individuo):
        if(self.p_mutacion>random()):
            indice=randint(0,15)
            nuevo_valor=uniform(self.min_poblacion[indice],self.max_poblacion[indice])
            individuo[indice]=nuevo_valor

    def _Cruce(self,pais):
        self._hijos=[]
        while(len(self._hijos))<self.tamaño_poblacion:
            _nuevo_individuo=[]
            self._NuevaRuleta(pais)
            if(self.p_cruce<random()): #Usamos nuestro p-cruce
                mitad=len(self._padre)//2
                for indice in range(len(self._padre)):
                    if(indice<mitad):
                        _nuevo_individuo.append(self._padre[indice])
                    else:
                        _nuevo_individuo.append(self._madre[indice])
# ------------- ERROR -----------------------------------------

                self._hijos.append(_nuevo_individuo)
            else:
                self._hijos.append(self._padre)

    def _Funcion_Objetivo(self,individuo): #16 parametros #32
        return list(self.modelo.predict(np.matrix(individuo)))


    def _NuevaRuleta(self,pais): #SELECCION DE RULETA
        """Selecciona un padre y una madre según las reglas de la ruleta."""
        def sortear(fitness_total, indice_a_ignorar=-1):
            #2 parámetros garantizan que no seleccionará el mismo elemento
            """Configurar la ruleta para realizar el sorteo"""
            ruleta, acumulado, valor_sorteado = [], 0, random()

            if indice_a_ignorar!=-1: #Deduce del total, la cantidad que se retirará de la ruleta
                fitness_total -= valores[0][indice_a_ignorar]
                # L = [123,456,789,132]
                #       0   1   2   3   -1
            for indice, i in enumerate(valores[0]):
                if indice_a_ignorar==indice: #ignorar el valor ya utilizado en la ruleta
                    continue
                acumulado += i
                ruleta.append(acumulado/fitness_total)
                if ruleta[-1] >= valor_sorteado:
                    return indice
        
        valores = list(zip(*pais)) #crea 2 listas con valores de fitness y cromosomas
        fitness_total = sum(valores[0])
        # [ (314,316,  )([1.01,30.1,])  ]
        indice_padre = sortear(fitness_total) 
        indice_madre = sortear(fitness_total, indice_padre)

        self._padre = valores[1][indice_padre]
        self._madre = valores[1][indice_madre]

    def _evolucion(self,poblacion:list):
        pais = [[self._Fitness(x),x] for x in poblacion]
        pais.sort()
        aux=pais
        for i in aux:
            print(i)
        self._Cruce(pais)
        #MUTACION
        for individuo in self._hijos:
            self._Mutacion(individuo)
        return self._hijos


In [ ]:
import os
N=100
Nuevo=AG(N)
Nuevo.modelo=model
Nuevo.p_cruce=0.8
Nuevo.p_mutacion=0.3
Nuevo._Poblacion()
PoblacionInicial=Nuevo.poblacion
os.system('cls')
for i in range(200):
    PoblacionInicial=Nuevo._evolucion(PoblacionInicial)
todo=PoblacionInicial
Optimo=Nuevo.poblacion[-1]
Final=Nuevo._Fitness(Optimo)
os.system('cls')
print("---------------------------------------------------------------")
print(Final)
print("---------------------------------------------------------------")

In [23]:
variable_x=[]
variable_y=[]
i=-1
while True:
    if(i==-10):
        break
    y=Nuevo._Fitness(todo[i])
    variable_x.append(todo[i])
    variable_y.append(y)
    i-=1
print(Matrix)

c:\Users\Edward\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
c:\Users\Edward\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


ValueError: X has 17 features, but SVC is expecting 16 features as input.